In [22]:
today_date = ''

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 24, Finished, Available, Finished)

In [2]:
from pyspark.sql.functions import col, count


StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 4, Finished, Available, Finished)

In [3]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 5, Finished, Available, Finished)

**Creating dim_location table**

In [4]:
df_lookup = (
    spark.read.format("csv")
    .option("header", "true")
    .load("Files/Landing/Taxilookup/taxi_zone_lookup.csv")
    .withColumn("location_id", col("LocationID").cast("int"))
    .drop("LocationID")
    .withColumnRenamed("Borough", "borough")
    .withColumnRenamed("Zone", "zone")
    .withColumnRenamed("service_zone", "service_zone")
)

display(df_lookup)

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a5753460-2d87-4340-8ed5-1fc3278d7ad6)

In [5]:
%%sql

CREATE TABLE IF NOT EXISTS dim_location (
   location_id INT,
   borough STRING,
   zone STRING,
   service_zone STRING 
)

USING DELTA

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

**Adding Row_number to dim_location and naming as surrogate key**

In [6]:
dim_location_df = (
    df_lookup.select("location_id", "borough", "zone", "service_zone") \
    .dropDuplicates()
)

window_spec = Window.orderBy("location_id")

dim_location_with_sk = dim_location_df.withColumn(
    "location_sk",
    row_number().over(window_spec)
)

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 8, Finished, Available, Finished)

In [7]:
dim_location_with_sk.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeschema", "true") \
    .saveAsTable("dim_location")

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 9, Finished, Available, Finished)

**Integrating yellow Nyc taxi and green Nyc taxi**

In [8]:
from pyspark.sql.functions import col, to_date, col, trim, regexp_replace, when, expr, to_timestamp, date_format, lit, isnull, hour, concat_ws, sha2

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 10, Finished, Available, Finished)

**Loading yellowtaxi_silver**

In [9]:
df_yellow = spark.read.format('delta').load('abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Tables/yellowtaxi_silver').filter(col("Processing_Date")==str(today_date))

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 11, Finished, Available, Finished)

**Loading greentaxi_silver**

In [10]:
df_green = spark.read.format('delta').load('abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Tables/greentaxi_silver').filter(col("Processing_Date")==str(today_date))

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 12, Finished, Available, Finished)

**Combining Yellow and Green Nyx taxi data**

In [11]:
fact_trip = df_green.unionAll(df_yellow)

print('Count of rows', fact_trip.count())

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 13, Finished, Available, Finished)

Count of rows 4336696


**Calling dim_location for join with fact_trip**

In [12]:
dim_location = spark.sql("SELECT * FROM NycTaxiProject.dim_location LIMIT 1000")
display(dim_location )

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2cd6d39a-1bdd-4cb6-9f69-d4516c49ab36)

**Retrieve Pickup_location_sk using pickup_location_id. Replace pickup_location_id with pickup_location_sk - in fact table**

In [13]:
fact_trip_new = fact_trip \
    .join(
        dim_location.select("location_sk", "location_id", "borough", "zone", "service_zone"),
        fact_trip.pickup_location_id == dim_location.location_id,
        "left"
    ) \
    .withColumnRenamed("location_sk", "pickup_location_sk") \
    .withColumnRenamed("borough", "pickup_borough") \
    .withColumnRenamed("zone", "pickup_zone") \
    .withColumnRenamed("service_zone", "pickup_service_zone") \
    .drop("location_id")

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 15, Finished, Available, Finished)

**Retrieve dropoff_location_sk using dropoff_location_id. Replace dropoff_location_id with dropoff_location_sk - in fact table**

In [14]:
fact_trip = fact_trip_new \
    .join(
        dim_location.select("location_sk", "location_id", "borough", "zone", "service_zone"),
        fact_trip_new.dropoff_location_id == dim_location.location_id,
        "left"
    ) \
    .withColumnRenamed("location_sk", "dropoff_location_sk") \
    .withColumnRenamed("borough", "dropoff_borough") \
    .withColumnRenamed("zone", "dropoff_zone") \
    .withColumnRenamed("service_zone", "dropoff_service_zone") \
    .drop("location_id")

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 16, Finished, Available, Finished)

**Concat pickup and dropoff locations**

In [15]:
fact_trip = (fact_trip.withColumn(
    "pickup_dropoff_borough",
    concat_ws(
        " - ",
        col("pickup_borough"),
        col("dropoff_borough")
    )
) \
.withColumn(
    "pickup_dropoff_zone",
    concat_ws(
        " - ",
        col("pickup_zone"),
        col("dropoff_zone")
    )
) \
.withColumn(
    "pickup_dropoff_service_zone",
    concat_ws(
        " - ",
        col("pickup_service_zone"),
        col("dropoff_service_zone")
    )
) \
.drop(
    "pickup_borough", "dropoff_borough",
    "pickup_zone", "dropoff_zone",
    "pickup_service_zone", "dropoff_service_zone"

)
)

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 17, Finished, Available, Finished)

**Removing duplication after integration**

In [16]:
print("before dropping duplicates", fact_trip.count())
fact_trip = fact_trip.dropDuplicates(["trip_natural_key"])
print("after dropping duplicates", fact_trip.count())

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 18, Finished, Available, Finished)

before dropping duplicates 4336696
after dropping duplicates 4336696


In [17]:
fact_trip.columns

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 19, Finished, Available, Finished)

['vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'rate_code_id',
 'pickup_location_id',
 'dropoff_location_id',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'cbd_congestion_fee',
 'processing_date',
 'invalid_numeric_flag',
 'invalid_location_flag',
 'vendor_name',
 'payment_type_name',
 'final_rate_code_name',
 'trip_duration_minutes',
 'pickup_day_of_week',
 'pickup_week_day_type',
 'pickup_month',
 'pickup_month_year',
 'pickup_time_of_day',
 'pickup_time_of_day_range',
 'source_name',
 'date_dq_flag',
 'pickup_time_bucket',
 'pickup_time_string',
 'dropoff_time_string',
 'trip_business_key',
 'trip_natural_key',
 'pickup_location_sk',
 'dropoff_location_sk',
 'pickup_dropoff_borough',
 'pickup_dropoff_zone',
 'pickup_dropoff_service_zone']

**Creating the fact table**

In [18]:
## create nyc taxi fact table

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, DecimalType

from delta.tables import DeltaTable

nyctaxitrip_schema = StructType([
    StructField('vendor_id', IntegerType(), True),
    StructField('pickup_datetime', DateType(), True),
    StructField('dropoff_datetime', DateType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DecimalType(10, 2), True),
    StructField('rate_code_id', IntegerType(), True),
    StructField('pickup_location_id', IntegerType(), True),
    StructField('dropoff_location_id', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DecimalType(10, 2), True),
    StructField('extra', DecimalType(10, 2), True),
    StructField('mta_tax', DecimalType(10, 2), True),
    StructField('tip_amount', DecimalType(10, 2), True),
    StructField('tolls_amount', DecimalType(10, 2), True),
    StructField('improvement_surcharge', DecimalType(10, 2), True),
    StructField('total_amount', DecimalType(10, 2), True),
    StructField('congestion_surcharge', DecimalType(10, 2), True),
    StructField('cbd_congestion_fee', DecimalType(10, 2), True),
    StructField('processing_date', DateType(), True),
    StructField('vendor_name', StringType(), True),
    StructField('payment_type_name', StringType(), True),
    StructField('final_rate_code_name', StringType(), True),
    StructField('trip_duration_minutes', IntegerType(), True),
    StructField('pickup_day_of_week', StringType(), True),
    StructField('pickup_week_day_type', StringType(), True),
    StructField('pickup_month', StringType(), True),
    StructField('pickup_month_year', StringType(), True),
    StructField('pickup_time_of_day', IntegerType(), True),
    StructField('pickup_time_of_day_range', StringType(), True),
    StructField('source_name', StringType(), True),
    StructField('pickup_time_bucket', StringType(), True),
    StructField('pickup_time_string', StringType(), True),
    StructField('dropoff_time_string', StringType(), True),
    StructField('trip_business_key', StringType(), True),
    StructField('trip_natural_key', StringType(), True),
    StructField('pickup_location_sk', IntegerType(), True),
    StructField('dropoff_location_sk', IntegerType(), True),
    StructField('pickup_dropoff_borough', StringType(), True),
    StructField('pickup_dropoff_zone', StringType(), True),
    StructField('pickup_dropoff_service_zone', StringType(), True),
])

DeltaTable.createIfNotExists(spark).tableName('fact_nyc_taxi').addColumns(nyctaxitrip_schema).execute()

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 20, Finished, Available, Finished)

**Loading data into fact_nyc_taxi**

In [19]:
fact_trip_selected_fact_nyc_taxi = (fact_trip.select(
'vendor_id',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'rate_code_id',
 'pickup_location_id',
 'dropoff_location_id',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'cbd_congestion_fee',
 'processing_date',
 'vendor_name',
 'payment_type_name',
 'final_rate_code_name',
 'trip_duration_minutes',
 'pickup_day_of_week',
 'pickup_week_day_type',
 'pickup_month',
 'pickup_month_year',
 'pickup_time_of_day',
 'pickup_time_of_day_range',
 'source_name',
 'pickup_time_bucket',
 'pickup_time_string',
 'dropoff_time_string',
 'trip_business_key',
 'trip_natural_key',
 'pickup_location_sk',
 'dropoff_location_sk',
 'pickup_dropoff_borough',
 'pickup_dropoff_zone',
 'pickup_dropoff_service_zone').dropDuplicates()
)

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 21, Finished, Available, Finished)

In [20]:
from delta.tables import *

fact_nyc_taxi_table_path = 'abfss://NycTaxiProject@onelake.dfs.fabric.microsoft.com/NycTaxiProject.Lakehouse/Tables/fact_nyc_taxi'
fact_deltanyctaxi = DeltaTable.forPath(spark,fact_nyc_taxi_table_path)

## Perform the Merge

fact_deltanyctaxi.alias('target').merge(
    fact_trip_selected_fact_nyc_taxi.alias('source'), 'target.trip_natural_key = source.trip_natural_key'
).whenMatchedUpdate( set = {
'vendor_id' : 'source.vendor_id',
'pickup_datetime' : 'source.pickup_datetime',
'dropoff_datetime' : 'source.dropoff_datetime',
'passenger_count' : 'source.passenger_count',
'trip_distance' : 'source.trip_distance',
'rate_code_id' : 'source.rate_code_id',
'pickup_location_id' : 'source.pickup_location_id',
'dropoff_location_id' : 'source.dropoff_location_id',
'payment_type' : 'source.payment_type',
'fare_amount' : 'source.fare_amount',
'extra' : 'source.extra',
'mta_tax' : 'source.mta_tax',
'tip_amount' : 'source.tip_amount',
'tolls_amount' : 'source.tolls_amount',
'improvement_surcharge' : 'source.improvement_surcharge',
'total_amount' : 'source.total_amount',
'congestion_surcharge' : 'source.congestion_surcharge',
'cbd_congestion_fee' : 'source.cbd_congestion_fee',
'processing_date' : 'source.processing_date',
'vendor_name' : 'source.vendor_name',
'payment_type_name' : 'source.payment_type_name',
'final_rate_code_name' : 'source.final_rate_code_name',
'trip_duration_minutes' : 'source.trip_duration_minutes',
'pickup_day_of_week' : 'source.pickup_day_of_week',
'pickup_week_day_type' : 'source.pickup_week_day_type',
'pickup_month' : 'source.pickup_month',
'pickup_month_year' : 'source.pickup_month_year',
'pickup_time_of_day' : 'source.pickup_time_of_day',
'pickup_time_of_day_range': 'source.pickup_time_of_day_range',
'source_name' : 'source.source_name',
'pickup_time_bucket' : 'source.pickup_time_bucket',
'pickup_time_string' : 'source.pickup_time_string',
'dropoff_time_string' : 'source.dropoff_time_string',
'trip_business_key' : 'source.trip_business_key',
'trip_natural_key' : 'source.trip_natural_key',
'pickup_location_sk' : 'source.pickup_location_sk',
'dropoff_location_sk' : 'source.dropoff_location_sk',
'pickup_dropoff_borough' : 'source.pickup_dropoff_borough',
'pickup_dropoff_zone' : 'source.pickup_dropoff_zone',
'pickup_dropoff_service_zone' : 'source.pickup_dropoff_service_zone'
}
).whenNotMatchedInsert(values={
'vendor_id' : 'source.vendor_id',
'pickup_datetime' : 'source.pickup_datetime',
'dropoff_datetime' : 'source.dropoff_datetime',
'passenger_count' : 'source.passenger_count',
'trip_distance' : 'source.trip_distance',
'rate_code_id' : 'source.rate_code_id',
'pickup_location_id' : 'source.pickup_location_id',
'dropoff_location_id' : 'source.dropoff_location_id',
'payment_type' : 'source.payment_type',
'fare_amount' : 'source.fare_amount',
'extra' : 'source.extra',
'mta_tax' : 'source.mta_tax',
'tip_amount' : 'source.tip_amount',
'tolls_amount' : 'source.tolls_amount',
'improvement_surcharge' : 'source.improvement_surcharge',
'total_amount' : 'source.total_amount',
'congestion_surcharge' : 'source.congestion_surcharge',
'cbd_congestion_fee' : 'source.cbd_congestion_fee',
'processing_date' : 'source.processing_date',
'vendor_name' : 'source.vendor_name',
'payment_type_name' : 'source.payment_type_name',
'final_rate_code_name' : 'source.final_rate_code_name',
'trip_duration_minutes' : 'source.trip_duration_minutes',
'pickup_day_of_week' : 'source.pickup_day_of_week',
'pickup_week_day_type' : 'source.pickup_week_day_type',
'pickup_month' : 'source.pickup_month',
'pickup_month_year' : 'source.pickup_month_year',
'pickup_time_of_day' : 'source.pickup_time_of_day',
'pickup_time_of_day_range': 'source.pickup_time_of_day_range',
'source_name' : 'source.source_name',
'pickup_time_bucket' : 'source.pickup_time_bucket',
'pickup_time_string' : 'source.pickup_time_string',
'dropoff_time_string' : 'source.dropoff_time_string',
'trip_business_key' : 'source.trip_business_key',
'trip_natural_key' : 'source.trip_natural_key',
'pickup_location_sk' : 'source.pickup_location_sk',
'dropoff_location_sk' : 'source.dropoff_location_sk',
'pickup_dropoff_borough' : 'source.pickup_dropoff_borough',
'pickup_dropoff_zone' : 'source.pickup_dropoff_zone',
'pickup_dropoff_service_zone' : 'source.pickup_dropoff_service_zone'
    
}).execute()

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 22, Finished, Available, Finished)

In [21]:
history_df = fact_deltanyctaxi.history(1)
operation_metrics = history_df.select("operationMetrics").collect()[0][0]
rows_inserted = operation_metrics.get('numTargetRowsInserted', 0)
rows_updated = operation_metrics.get('numTargetRowsUpdated', 0)
rows_deleted = operation_metrics.get('numTargetRowsDeleted', 0)
rows_affected = int(rows_inserted) + int(rows_updated) + int(rows_deleted)

print('Total rows of table: ', fact_deltanyctaxi.toDF().count())
print(f"Rows inserted: {rows_inserted}")
print(f"Rows updated: {rows_updated}")
print(f"Rows deleted: {rows_deleted}")
print(f"Total rows affected: {rows_affected}")

StatementMeta(, 62e1349b-4d46-4699-b6ad-61eebaee5dc4, 23, Finished, Available, Finished)

Total rows of table:  26774800
Rows inserted: 4336696
Rows updated: 0
Rows deleted: 0
Total rows affected: 4336696
